In [59]:
import  os
import  pandas as pd
from    tqdm import tqdm
import  numpy as np
from    multiprocessing import get_context as context
from    matplotlib import pyplot as plt
%matplotlib inline

# Read the files

# Parameters


header      = ['EventID','TrackID','Particle','EnergyDeposited','XPosition','YPosition','ZPosition','LocalTime','Volume', 'Initial Energy', 'Origin Volume', 'MigrantID']
PROCESSES   = os.cpu_count()-1
Pool        = context("fork").Pool

# Find the files
foldername  = f'../build/output'
os.system(f'rm -rf ../build/output/.ipynb_checkpoints')
filenames   = os.listdir(foldername)

print(f'Found {len(filenames)} data files in {foldername}')

def parallel(function):
    def inner(input_array):
        # Parallelize excecution them
        with Pool(PROCESSES) as pool:
            output = list(tqdm(pool.imap(function, input_array),total=len(input_array)))
        return output 
    return inner

# Function to load a single file
def loadfile(filename):
    df = pd.read_csv(os.path.join(foldername,filename),skiprows=16,names=header)
    df.insert(0, 'Filename', filename)
    return df

Found 19 data files in ../build/output


In [60]:
def processFile(filename):
    
    veto = loadfile(filename)
    veto = veto[veto.Volume.str.contains('veto')]
    veto = veto.groupby(['Filename','EventID', 'Volume'])['EnergyDeposited'].sum().reset_index()
    veto = veto[veto.EnergyDeposited > 10**(-9)]


    # loading the rest of the dataframe
    data = loadfile(filename)
    data = data[~(data.Volume.str.contains('veto'))]
    data = data.groupby(['Filename','EventID', 'Volume'])['EnergyDeposited'].sum().reset_index() # sum the energy per volume, hit and eventid are overlapping perfectly
    data = data[data['EnergyDeposited'] > 10**(-9)]

    return veto,data


processFiles        = parallel(processFile)
processed           = processFiles(filenames)

100%|██████████| 19/19 [00:08<00:00,  2.32it/s]


In [61]:
vetos = [i[0] for i in processed]
datas = [i[1] for i in processed]

In [62]:
veto = pd.concat(vetos)
data = pd.concat(datas)

In [63]:
veto = veto.rename(columns={'EnergyDeposited':'vetoEnergy'})
veto = veto.drop(columns='Volume')

In [64]:
crystal1 = data[data['Volume'] == 'crystal1_PV']
crystal2 = data[data['Volume'] == 'crystal2_PV']
crystal3 = data[data['Volume'] == 'crystal3_PV']
crystal4 = data[data['Volume'] == 'crystal4_PV']
crystal1_2 = data[data['Volume'].isin(['crystal1_PV','crystal2_PV'])]
crystal1_2_3_4 = data[data['Volume'].isin(['crystal1_PV','crystal2_PV','crystal3_PV','crystal4_PV'])]

In [65]:
# Veto + crystal1 coincidence

def coincidence(data1: pd.DataFrame, data2: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes in dataframes that have only one volume.
    It merges them based on their EventID and sums their energies
    """
    combined = pd.DataFrame.merge(data1, data2, how = 'inner', on=['Filename','EventID'])
    if 'EnergyDeposited_x' in combined.columns:
        combined['EnergyDeposited'] = combined['EnergyDeposited_x'] + combined['EnergyDeposited_y']
    columns_to_drop = ['EnergyDeposited_x','EnergyDeposited_y', 'Volume_x', 'Volume_y','Volume']
    for column in columns_to_drop:
        if column in combined.columns:
            combined = combined.drop(columns=column)
    return combined

def union(data1: pd.DataFrame, data2: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes in two dataframes that have only one volume
    It concats them based on their EventID as a union
    """

    combined = pd.concat([data1,data2])
    return combined

In [66]:
veto_and_1 = coincidence(veto,crystal1)


In [67]:
veto_and_1

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,399,1.593318,4.335682
1,TES-run0-rank8_nt_hits.csv,3523,2.132889,13.106900
2,TES-run0-rank8_nt_hits.csv,10573,0.896514,22.634428
3,TES-run0-rank8_nt_hits.csv,13175,0.033150,6.458969
4,TES-run0-rank8_nt_hits.csv,14072,1.998085,15.040005
...,...,...,...,...
58441,TES-run0-rank4_nt_hits.csv,5249931,0.956616,18.539232
58442,TES-run0-rank4_nt_hits.csv,5254182,4.419966,1.365230
58443,TES-run0-rank4_nt_hits.csv,5258218,2.259154,19.533163
58444,TES-run0-rank4_nt_hits.csv,5258374,1.261485,1.027990


In [68]:
veto_and_1or2 = coincidence(veto,crystal1_2)
veto_and_1or2

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,399,1.593318,4.335682
1,TES-run0-rank8_nt_hits.csv,399,1.593318,19.365145
2,TES-run0-rank8_nt_hits.csv,422,1.009450,0.984708
3,TES-run0-rank8_nt_hits.csv,2481,1.831872,16.305700
4,TES-run0-rank8_nt_hits.csv,3523,2.132889,13.106900
...,...,...,...,...
116452,TES-run0-rank4_nt_hits.csv,5258218,2.259154,0.881502
116453,TES-run0-rank4_nt_hits.csv,5258374,1.261485,1.027990
116454,TES-run0-rank4_nt_hits.csv,5259515,1.121540,11.506700
116455,TES-run0-rank4_nt_hits.csv,5261112,1.981504,0.047104


In [69]:
veto_and_1or2or3or4 = coincidence(veto,crystal1_2_3_4)
veto_and_1or2or3or4

,Filename,EventID,vetoEnergy,EnergyDeposited
0,TES-run0-rank8_nt_hits.csv,68,0.855991,3.651216
1,TES-run0-rank8_nt_hits.csv,399,1.593318,4.335682
2,TES-run0-rank8_nt_hits.csv,399,1.593318,19.365145
3,TES-run0-rank8_nt_hits.csv,399,1.593318,0.328699
4,TES-run0-rank8_nt_hits.csv,422,1.009450,0.984708
...,...,...,...,...
235441,TES-run0-rank4_nt_hits.csv,5260302,2.152235,17.020455
235442,TES-run0-rank4_nt_hits.csv,5261112,1.981504,0.047104
235443,TES-run0-rank4_nt_hits.csv,5262373,0.960517,14.168593
235444,TES-run0-rank4_nt_hits.csv,5263102,0.968185,0.225366


In [77]:
def message(value, text):
    percent = round(value*100,2)
    print(f'{text} = {percent}%')

In [78]:
# Veto and crystal1 
value = len(veto_and_1)/len(veto)
message(value, 'Veto and crystal1 / veto')

Veto and crystal1 / veto = 23.47%


In [79]:
# (Not veto) and crystal1
value = ((len(crystal1) - len(veto_and_1))/(len(crystal1)))
message(value, 'Crystal1 and (not Veto) / (Crystal1)')

Crystal1 and (not Veto) / (Crystal1) = 23.48%


In [73]:
# Veto and (crystal1 or crystal2)
value = len(veto_and_1or2)/len(veto)
message(value, 'Veto and (Crystal1 or Crystal2) / veto')

Veto and (Crystal1 or Crystal2) / veto = 46.8%


In [74]:
# (Not veto) and (crystal1 or crytal2)

value = (len(crystal1_2)-len(veto_and_1or2))/(len(crystal1_2))
percent = round(value*100,1)
print(f'(Crystal1 or Crystal2) and (not Veto) / (Crystal1 or Crystal2) = {percent}%')

(Crystal1 or Crystal2) and (not Veto) / (Crystal1 or Crystal2) = 23.4%


In [75]:
# Veto and union(1,2,3,4) / veto

value = (len(veto_and_1or2or3or4)/len(veto))

message(value, 'Union(1,2,3,4) and veto / veto')

Union(1,2,3,4) and veto / veto = 94.5%


In [76]:
# (Not veto) and union(1,2,3,4)

value = (len(crystal1_2_3_4)-len(veto_and_1or2or3or4))/(len(crystal1_2_3_4))
percent = round(value*100,1)
print(f'Union(1,2,3,4) and (not Veto) / Union(1,2,3,4) = {percent}%')

Union(1,2,3,4) and (not Veto) / Union(1,2,3,4) = 24.1%
